# Understanding kinyu.graph: Caching and Dependency Tracking

This notebook demonstrates the core features of the `kinyu.graph` module. The `@g_func` decorator provides a powerful way to build a dependency graph of your functions, enabling automatic caching and intelligent re-computation.

## Caching for Performance

One of the primary benefits of `kinyu.graph` is its caching capability. When you decorate a function with `@g_func`, the result of its first execution is stored. On subsequent calls with the same arguments, the cached result is returned instantly, avoiding expensive re-computation. This is particularly useful for functions that perform time-consuming tasks like I/O operations or complex numerical calculations.

## Dependency Tracking for Transparency

`@g_func` automatically tracks the relationships between your functions. If `a()` calls `b()`, the graph registers that `a` depends on `b`. This dependency graph provides transparency into your calculation logic.

More importantly, the graph uses this information to manage the cache intelligently. If you override the value of a node (e.g., `b`), the graph knows that all its ancestors (e.g., `a`) are now "stale" and must be re-executed on their next call. The cache for unrelated functions or descendants remains valid, ensuring maximal efficiency.

In [1]:
import time
import pprint
import sys

# Add the src directory to the path to find the kinyu module
sys.path.insert(0, '../src')

from kinyu.graph import g_func, get_edges, override_value, clear_cache

# We clear the cache at the start of each demonstration
clear_cache()

## Use Case 1: Standalone Functions

This example demonstrates the basic usage of `@g_func` with simple, standalone functions. We'll create a dependency chain `a -> b -> c`, where `c` is an expensive operation.

In [2]:
@g_func
def a():
    print("Executing a()...")
    return b() * 2

@g_func
def b():
    print("Executing b()...")
    return c() + 10

@g_func
def c():
    print("Executing c() (expensive operation)...")
    time.sleep(0.5)
    return 5

print("--- First execution of a() ---")
start_time = time.time()
result = a()
duration = time.time() - start_time
print(f"Result: {result}, Duration: {duration:.2f}s\n")

print("--- Second execution of a() (from cache) ---")
start_time = time.time()
result = a()
duration = time.time() - start_time
print(f"Result: {result}, Duration: {duration:.2f}s\n")

print("--- Dependency Graph Edges ---")
pprint.pprint(get_edges())
print("")

print("--- Overriding c() and re-executing a() ---")
override_value('c', 100)
result = a()
print(f"New Result: {result}")

--- First execution of a() ---
Executing a()...
Executing b()...
Executing c() (expensive operation)...


Result: 30, Duration: 0.51s

--- Second execution of a() (from cache) ---
Result: 30, Duration: 0.00s

--- Dependency Graph Edges ---
[('b(*(), **frozendict.frozendict({}))', 'c(*(), **frozendict.frozendict({}))'),
 ('a(*(), **frozendict.frozendict({}))', 'b(*(), **frozendict.frozendict({}))')]

--- Overriding c() and re-executing a() ---
Executing a()...
Executing b()...
New Result: 220


## Use Case 2: Instance Methods

The `@g_func` decorator is also aware of object instances. When applied to a method, the identity of the instance (`self`) is included in the cache key. This means that calls to the same method on *different objects* are treated as distinct nodes in the graph.

This is crucial for object-oriented designs where multiple instances of a class perform similar but independent calculations.

In [3]:
class MyCalculator:
    def __init__(self, name, base_value):
        self.name = name
        self.base_value = base_value
        print(f"Created {self.name} with base value {self.base_value}")

    @g_func
    def intermediate_step(self):
        print(f"Executing intermediate_step() for {self.name}...")
        return self.expensive_operation() + self.base_value

    @g_func
    def final_result(self):
        print(f"Executing final_result() for {self.name}...")
        return self.intermediate_step() * 2

    @g_func
    def expensive_operation(self):
        print(f"Executing expensive_operation() for {self.name}...")
        time.sleep(0.5)
        return 10

# Clear cache to start fresh
clear_cache()

# Create two independent calculator instances
calc1 = MyCalculator("Calculator_1", 100)
calc2 = MyCalculator("Calculator_2", 500)
print("")

print("--- Executing on Calculator_1 ---")
result1 = calc1.final_result()
print(f"Result for {calc1.name}: {result1}\n")

print("--- Executing on Calculator_2 ---")
result2 = calc2.final_result()
print(f"Result for {calc2.name}: {result2}\n")

print("--- Re-executing on Calculator_1 (should be cached) ---")
result1_cached = calc1.final_result()
print(f"Cached result for {calc1.name}: {result1_cached}\n")

print("--- Overriding a value on Calculator_2 only ---")
override_value('expensive_operation', 1000, instance=calc2)
new_result2 = calc2.final_result()
print(f"New result for {calc2.name} after override: {new_result2}\n")

print("--- Final Dependency Graph ---")
pprint.pprint(get_edges())

Created Calculator_1 with base value 100
Created Calculator_2 with base value 500

--- Executing on Calculator_1 ---
Executing final_result() for Calculator_1...
Executing intermediate_step() for Calculator_1...
Executing expensive_operation() for Calculator_1...


Result for Calculator_1: 220

--- Executing on Calculator_2 ---
Executing final_result() for Calculator_2...
Executing intermediate_step() for Calculator_2...
Executing expensive_operation() for Calculator_2...


Result for Calculator_2: 1020

--- Re-executing on Calculator_1 (should be cached) ---
Cached result for Calculator_1: 220

--- Overriding a value on Calculator_2 only ---
Executing final_result() for Calculator_2...
Executing intermediate_step() for Calculator_2...
New result for Calculator_2 after override: 3000

--- Final Dependency Graph ---
[('intermediate_step[0x7f8f046acec0](*(), **frozendict.frozendict({}))',
  'expensive_operation[0x7f8f046acec0](*(), **frozendict.frozendict({}))'),
 ('final_result[0x7f8f046acec0](*(), **frozendict.frozendict({}))',
  'intermediate_step[0x7f8f046acec0](*(), **frozendict.frozendict({}))'),
 ('final_result[0x7f8f046ace30](*(), **frozendict.frozendict({}))',
  'intermediate_step[0x7f8f046ace30](*(), **frozendict.frozendict({}))'),
 ('intermediate_step[0x7f8f046ace30](*(), **frozendict.frozendict({}))',
  'expensive_operation[0x7f8f046ace30](*(), **frozendict.frozendict({}))')]
